In [ ]:
# dataset preparation
from score_topology.data import *


file = open('./test/test.json', 'rb')
data = loadConnectionSet(file)

#seqs, matrixH, masks = exampleToTensors(data['connections'][0], 0x100, 0x200)
#print(seqs, matrixH, masks)

examples = list(map(lambda ex: exampleToTensors(ex, 0x100, 0x200), data['connections']))
dataset = batchizeTensorExamples(examples, 4)

print('dataset:', dataset[0]['seq_id'].shape, dataset[0]['seq_position'].shape, dataset[0]['mask'].shape, dataset[0]['matrixH'].shape)
#print('seq_id.0', dataset[0]['seq_id'][0])
#print('seq_position.0', dataset[0]['seq_position'][0][1])


In [ ]:
# sequence masking
import torch
from transformer.models import get_subsequent_mask, get_pad_mask


seq = torch.tensor([[3,2,1], [4,5,6]])
mask1 = get_pad_mask(seq, 1)
mask2 = get_subsequent_mask(seq)
print(mask1)
print(mask2)
print(mask1 & mask2)


In [ ]:
# model test
from starry.score_topology.models import *


model = TransformJointer()
batch = dataset[0]
pred = model(batch['seq_id'], batch['seq_position'], batch['mask'])


In [ ]:
import torch


t1 = torch.tensor([1,2,3])
t2 = torch.tensor([4,5,6])

t1 = t1.unsqueeze(0).repeat(2, 1, 1)
t2 = t2.unsqueeze(1)
print(t1, t2)

p = t1.matmul(t2)
print(p)


In [ ]:
# predictor test
from ipywidgets import interact, interact_manual
import json
from starry.utils.config import Configuration
from starry.score_topology.data import *
from starry.score_topology.predictor import *


def test (config, clusters):
	predictor = Predictor(config)
	results = predictor.predict(clusters)
	text = json.dumps(results)

	print('results:', text)


def setConfig(config_dir, topology_file):
	config = Configuration(config_dir)
	print('config loaded:', config.id)

	file = open(topology_file, 'r')
	data = loadClusterSet(file)
	clusters = data.get('clusters', data['connections'])[:4]
	test(config, clusters)


interact_manual(setConfig, config_dir='', topology_file='')
